# Test LLL and LL fault for all test cases

This Notebook shows exemplary tests that are implemented in the sce_test/test_all_faults_and_cases.py file.

See: https://github.com/e2nIEE/pandapower/blob/sce/pandapower/test/shortcircuit/SCE_Tests/test_all_faults_and_cases.py

Therefore it must be run in an environment where the pandapower test folders can be found by the jupyter server's python environment.
This can be done by checking out the git branch sce in pandapower and locally initialize the repository by using pip install -e.

Further information on running short-circuit calculations in pandapower can be found here:

https://pandapower.readthedocs.io/en/latest/shortcircuit/run.html

https://github.com/e2nIEE/pandapower/blob/sce/tutorials/shortcircuit/shortcircuit.ipynb

In [38]:
import os
from itertools import product
import itertools
import numpy as np

from pandapower import pp_dir
from pandapower.file_io import from_json
from pandapower.test.shortcircuit.sce_tests.test_all_faults_and_cases import load_pf_results, modify_impedance_values_with_fault_value, check_pattern, load_test_case_data, run_test_cases, compare_results
from pandapower.shortcircuit import calc_sc

testfiles_path = os.path.join(pp_dir, 'test', 'shortcircuit', 'sce_tests')

In [39]:
import warnings
# Ignore all future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all runtime warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

From the defined parameters for faults, cases, network names and vector_groups as well as the values for r_faults and x_faults all combinations are generated to run as a test.

In [40]:
# Define common parameters
net_names = [
    "test_case_2_five_bus_radial_grid",
    "test_case_3_five_bus_meshed_grid",
    "test_case_4_twenty_bus_radial_grid"
]
faults = ["LLL", "LL"]
cases = ["max", "min"]
values = [(0.0, 0.0), (5.0, 5.0)]
vector_groups = ['Dyn', 'Yyn', 'YNyn']
lv_tol_percents = [6, 10]
fault_location_buses = [0, 1, 2, 3]
is_branch_test = [False, True]

# Create parameter list
parametrize_values = list(product(faults, cases, values, lv_tol_percents, fault_location_buses, is_branch_test))

# Create parameter list with vector group
parametrize_values_vector = list(product(net_names, faults, cases, values, lv_tol_percents, vector_groups, fault_location_buses, is_branch_test))

total_test_count = len(parametrize_values)+len(parametrize_values_vector)
formatted_output = (
    f"Number of parameterization values: {len(parametrize_values)}\n"
    f"Number of parameterization value vectors: {len(parametrize_values_vector)}\n"
    f"Total number of tests: {total_test_count}"
)
print(formatted_output)

Number of parameterization values: 128
Number of parameterization value vectors: 1152
Total number of tests: 1280


The `parametrize_values` list and the `parametrize_values_vector` contain combinations for the tests. There is one test using `parametrize_values` and three using `parametrize_values_vector`. Shown above are the amount of test for each of the lists as well as the total.

In this Notebook we will only show a exemplary examples for each test case and vector group.
The test will load the expected results from the corresponding excel file (exported from comemrcial powerfactory software results). Then the calculations are run in pandapower using the parmeters from the list of permutiation and compared against the expected results.
However if you run the tests in here, you will be able to execute all 1.280 passing tests.

https://github.com/e2nIEE/pandapower/blob/sce/pandapower/test/shortcircuit/SCE_Tests/test_all_faults_and_cases.py

---

## Test Case 1: Four bus radial grid

![four_bus_radial](four_bus_radial.png)

In [41]:
net_name = "test_case_1_four_bus_radial_grid"

fault = "LLL"
case = "max"
fault_values = (0.0, 0.0)
# lv_tol_percents does not influence max case (default value is 10)
lv_tol_percents = 10
fault_location_bus = 0
is_branch_test = False
# vector group not necesarry as the grid does not contain a transformer

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus)

These are all the test results that were exported from PowerFactory. Depending on the parameter combination, the corresponding dataframe is selected.

In [42]:
dataframes["bus"]["LLL_max_10"]

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163
1,Bus_1,0.000000,0.000000,0.000000,0.000000
2,Bus_2,0.000000,0.000000,0.000000,0.000000
3,Bus_3,0.000000,0.000000,0.000000,0.000000


Let's now select the dataframe that we need in order to execute our first test case.

With the loaded network and expected results we now can run the test to receive the dataframes and the columns that should be compared.

Below is the list of columns that will be compared.

In [43]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['ikss_ka', 'skss_mw', 'rk_ohm', 'xk_ohm'], dtype='object')

In [44]:
net_df

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886751,100.0,0.437816,4.378164


In [45]:
# dataframes["bus"]["LLL_max_10"]
pf_results

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163


As you can see the results are matching and the comparison function will pass and not throw any errors.

In [46]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)

## Test Case 2: Five bus radial grid

![five_bus_radial](five_bus_radial.png)

In [47]:
net_name = "test_case_2_five_bus_radial_grid"

fault = "LL"
case = "min"
fault_values = (5.0, 5.0)
lv_tol_percents = 6
fault_location_bus = 3
is_branch_test = True
vector_group = 'Dyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [48]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['ikss_from_ka', 'ikss_to_ka', 'p_from_mw', 'q_from_mvar', 'p_to_mw',
       'q_to_mvar', 'vm_from_pu', 'va_from_degree', 'vm_to_pu',
       'va_to_degree'],
      dtype='object')

Note: The IEC 60909 standard does not require a phase angle of the short-circuit current for a two-phase short-circuit, only the effective current value. The standard does not provide for a calculation of the complex current, therefore no angle for ikss_from_degree and ikss_to_degree.
Powerfactory uses a method that goes beyond the IEC 60909 standard to calculate the angles for LL, which might be implemented in pandapower as well.

In [49]:
net_df

,name,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,Line_0,1.308134,1.308134,NaN,1.308134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Line_1,1.308134,1.308134,NaN,1.308134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Line_2,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# dataframes["branch"]["LL_min_fault_6"]
pf_results

,name,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,Line_0,1.308134,1.308134,37.108863,1.308134,-142.891137,1.671931e+00,1.085394e+01,3.654599e-02,-9.844324e+00,0.860060,118.351903,0.859268,127.321567
1,Line_1,1.308134,1.308134,-142.891137,1.308134,37.108863,1.745023e+00,-8.834708e+00,-3.654598e-02,9.844324e+00,0.865151,138.282050,0.859268,127.321567
2,Line_2,0.000000,0.000000,0.000000,0.000000,0.000000,8.127625e-10,3.045348e-10,-8.127640e-10,-3.045379e-10,0.859268,127.321567,0.859268,127.321567


In [51]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)

## Test Case 3: Five bus meshed grid

![five_bus_meshed](five_bus_meshed.png)

In [52]:
net_name = "test_case_3_five_bus_meshed_grid"

fault = "LLL"
case = "min"
fault_values = (5.0, 5.0)
lv_tol_percents = 6
fault_location_bus = 2
is_branch_test = True
vector_group = 'Yyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [53]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['ikss_ka', 'ikss_from_ka', 'ikss_from_degree', 'ikss_to_ka',
       'ikss_to_degree', 'p_from_mw', 'q_from_mvar', 'p_to_mw', 'q_to_mvar',
       'vm_from_pu', 'va_from_degree', 'vm_to_pu', 'va_to_degree'],
      dtype='object')

In [54]:
net_df

,name,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,Line_0,1.146247,1.146247,-52.634345,1.146247,127.365655,23.643589,22.033819,-19.708246,-19.708246,0.81393,-9.652736,0.70193,-7.634345
1,Line_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.70193,-7.634345,0.70193,-7.634345
2,Line_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.70193,-7.634345,0.70193,-7.634345
3,Line_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.70193,-7.634345,0.70193,-7.634345


In [55]:
# dataframes["branch"]["LLL_min_fault_6"]
pf_results

,name,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,Line_0,1.146247,1.146247,-52.634345,1.146247,127.365655,23.643589,22.033819,-1.970825e+01,-1.970825e+01,0.81393,-9.652736,0.70193,-7.634345
1,Line_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.233007e-09,1.025774e-09,0.70193,-7.634345,0.70193,-7.634345
2,Line_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.233008e-09,1.025769e-09,0.70193,-7.634345,0.70193,-7.634345
3,Line_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.70193,-7.634345,0.70193,-7.634345


In [56]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)

## Test Case 4: Twenty bus radial grid

![twenty_bus_grid](twenty_bus_grid.png)

In [57]:
net_name = "test_case_4_twenty_bus_radial_grid"

fault = "LL"
case = "max"
fault_values = (0.0, 0.0)
lv_tol_percents = 10
fault_location_bus = 3
is_branch_test = False
vector_group = 'YNyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [58]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['ikss_ka', 'skss_mw', 'rk_ohm', 'xk_ohm'], dtype='object')

In [59]:
net_df

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
3,3,4.792197,1.10671,0.012035,0.044302


In [60]:
# dataframes["bus"]["LL_max_10"]
pf_results

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
13,3,4.792197,1.106711,0.012035,0.044302


In [61]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)